<a href="https://colab.research.google.com/github/nyp-sit/iti107-2024S2/blob/main/session-3/yolo8_custom_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection using YOLO



## Streaming

We can also do real-time detection on a video or camera steram.

The code below uses openCV library to display video in a window, and can only be run locally on a local laptop.

If you are training your model on Google Colab, you will download the exported OpenVINO model to a local PC. If you are training your model locally, then the exported model should already be on your local PC.

Run the following code to zip up the OpenVINO folder and download to local PC.


*Note: If you encountered error message "NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968", uncomment the following cell and run it.*


In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [19]:
!pip install ultralytics
!pip install comet_ml

In [20]:
import ultralytics
from ultralytics import YOLO
from PIL import Image

In [21]:
!pip install openvino

### Streaming and display video

In [22]:
# from ultralytics import YOLO
import cv2
import openvino

# Load the YOLO model
model = YOLO("iti107_assignment/train/weights/best_int8_openvino_model", task="detect")

# Open the video file
video_path = "./fire-protection-9074899G.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO inference on the frame on GPU Device 0
        results = model(frame, device="cpu")

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLO Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

Loading iti107_assignment\train\weights\best_int8_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...

0: 640x640 3 extinguisher-helmets, 3 fire extinguishers, 1 fire signage, 52.3ms
Speed: 8.0ms preprocess, 52.3ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 extinguisher-helmets, 4 fire extinguishers, 1 fire signage, 2 helmets, 46.8ms
Speed: 10.0ms preprocess, 46.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 extinguisher-helmets, 3 fire extinguishers, 1 fire signage, 46.2ms
Speed: 6.0ms preprocess, 46.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 extinguisher-helmets, 4 fire extinguishers, 3 fire signages, 2 helmets, 53.4ms
Speed: 9.7ms preprocess, 53.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 extinguisher-helmets, 4 fire extinguishers, 3 fire signages, 44.7ms
Speed: 5.0ms preprocess, 44.7ms infere

### Detect and write to a video file

In [23]:
# from ultralytics import YOLO
# import cv2
from tqdm import tqdm

def write_video(video_in_filepath, video_out_filepath, model):
    # Open the video file

    video_reader = cv2.VideoCapture(video_in_filepath)

    nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = video_reader.get(cv2.CAP_PROP_FPS)

    video_writer = cv2.VideoWriter(video_out_filepath,
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps,
                            (frame_w, frame_h))

    # Loop through the video frames
    for i in tqdm(range(nb_frames)):
        # Read a frame from the video
        success, frame = video_reader.read()

        if success:
            # Run YOLO inference on the frame on GPU Device 0
            # results = model(frame, device=0)
            results = model(frame, device='cpu')

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Write the annotated frame
            video_writer.write(annotated_frame)

    video_reader.release()
    video_writer.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


In [24]:
from pathlib import Path
import os

video_in_file = "fire-protection-9074899G.mp4"
basename = Path(video_in_file).stem
video_out_file = os.path.join(basename + '_detected-1' + '.mp4')
model = YOLO("iti107_assignment/train/weights/best_int8_openvino_model", task="detect")
write_video(video_in_file, video_out_file, model)

  0%|                                                                                         | 0/1315 [00:00<?, ?it/s]

Loading iti107_assignment\train\weights\best_int8_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...

0: 640x640 3 extinguisher-helmets, 3 fire extinguishers, 1 fire signage, 46.5ms
Speed: 3.0ms preprocess, 46.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


  0%|                                                                                 | 1/1315 [00:00<21:21,  1.03it/s]


0: 640x640 3 extinguisher-helmets, 4 fire extinguishers, 1 fire signage, 2 helmets, 74.4ms
Speed: 6.0ms preprocess, 74.4ms inference, 11.2ms postprocess per image at shape (1, 3, 640, 640)


  0%|                                                                                 | 2/1315 [00:01<10:43,  2.04it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▊| 1312/1315 [01:37<00:00, 14.60it/s]


0: 640x640 2 fire extinguishers, 1 helmet, 31.3ms
Speed: 0.0ms preprocess, 31.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 fire extinguishers, 1 helmet, 22.6ms
Speed: 3.6ms preprocess, 22.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


100%|█████████████████████████████████████████████████████████████████████████████▉| 1314/1315 [01:37<00:00, 14.54it/s]


0: 640x640 2 fire extinguishers, 1 helmet, 21.5ms
Speed: 4.0ms preprocess, 21.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████████████████████████████████████████████████████████████████████████| 1315/1315 [01:37<00:00, 13.43it/s]
